In [1]:
#if u are facing computation problems then use google colab,,which provide online resources(gpu,tpu,cpu,ram..etc)
#to train model fastly

In [2]:
import zipfile
zip_ref=zipfile.ZipFile(r"D:\DEEP_LEARNING\C_N_N\4_cats_vs_dog_classification\archive.zip",'r')
zip_ref.extractall()#extracting all contents of archive.zip in cwd
zip_ref.close()

In [3]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Conv2D,MaxPooling2D,Flatten

In [4]:
#now we have to give i/p to cnn ie;train data,,,
#1 way is that(manually)(customize code),,using os model go inside train data folder and go inside cat and dog folder,then
#fetch the filename and give to cnn,,,and bcz images are more(nearly 20k), passing all these fetched cat and dog image from
#training data is become tidious task bcz of  ram shortage ,gpu shortage


In [5]:
#keras provides the Generator concept
#which generally divide the train data into multiple batches,,and at a time 1 batch is loaded into ram after processing
#on that batch is done then it is removed from ram and next batch is loaded,,,and so on untill all batches processing completes..
#after completion finally get optimum value for filters(w) and bias for each filter
#now u can test data

#therefore Genrator are used to process large amount of data,,if u have shortage of ram,gpu,tpu..etc

In [6]:
#Generator
train_ds=keras.utils.image_dataset_from_directory(
            directory = r"D:\DEEP_LEARNING\C_N_N\4_cats_vs_dog_classification\train",
            labels='inferred',
            label_mode='int',
            batch_size=32,
            image_size=(256,256)
)
#infer-->tarkane(not date) anuman kadne(marathi)/suchvne(denote)
#ds means dataset->for our notation
#label_mode='int'  cat folder  is 1st in train folder so -->0,,dog is 2nd folder so -->1
#batch_size=32,-->at a time 32 images are loaded into ram and given as i/p to cnn
#image_size=(256,256)--->bcz,all images are in diff resolution,,,thus training is not occurs good,,so make them in one size
#ie;256,256 and keras automatically understand that images are RGB(colorfull) so,internally use 3 channels(256x256x3) for each image

validation_ds=keras.utils.image_dataset_from_directory(
            directory = r"D:\DEEP_LEARNING\C_N_N\4_cats_vs_dog_classification\test",
            labels='inferred',
            label_mode='int',
            batch_size=32,
            image_size=(256,256)
)

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.


In [7]:
# Found 20000 files belonging to 2 classes.-->train_ds,,,1class is cat(0) and another class is dog(1)
#20k images are there intotal,,cat images are around 10k and dog=10k
# Found 5000 files belonging to 2 classes.-->test dataset(validation_ds),,cat(0),,dog(1)

In [8]:
#20k images(pixel) value stored in train_ds in the form of numpy array,,ie;colorfull image so 3d array used
#[]-->1d,,[[],[]]-->2d,,,[ [ [] , [] , [] ] , [ [] , [] , [] ] ]-->3d (2x3x3) 
#ie;256 rows and 256 cols and like this 3 matrix  are there(bcz RGB channel(3 channels))
#[[c1,c2,...c256],[c1,c2,...c256],........,[c1,c2,...c256]]  -->(256x256),,,grayscale image  for 1 image
#  r1             r2                       r256

#for RGB   ,,
#[ [ [c1,c2,...c256],[c1,c2,...c256],........,[c1,c2,...c256] ] ,[ [c1,c2,...c256],[c1,c2,...c256],........,[c1,c2,...c256] ] , [ [c1,c2,...c256],[c1,c2,...c256],........,[c1,c2,...c256] ]]
#  R  r1             r2                        r256              G  r1              r2                       r256               B  r1              r2                       r256
#(3x256x256)  for 1 image
#and such a 20k images are there in tain_ds thus it is represented within a LIST,,,,,or we can say 4d array
#(20000x3x256x256)-->20k images each with 3 cahnnels(matrix) and each matrix(channel) with 256 rows x 256 col

In [9]:
#and pixel values are 0 to 255 ,,,therefore squish them in range 0 to 1 and it should be float,,,,
#Normalization require

In [10]:
#Normalization
def process(image,label):
    image=tf.cast(image/255.,tf.float32)
    return image,label
train_ds=train_ds.map(process)#thus each value is 0 to 1 floating pt value
validation_ds=validation_ds.map(process)
#map will fetch one image and label from object and pass it to process func,,,repeats untill all images are processed
#now all pixel values are in b/n 0 to 1 floating value

# create CNN model

In [11]:
model=Sequential()#1 layer after another,,so need to mention i/p shape and type for each layer,,model will automatically INFER it,,,bcz it is sequential model

#C_L1
model.add(Conv2D(32,kernel_size=(3,3),padding='valid',activation='relu',input_shape=(256,256,3)))
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))

#C_L2
model.add(Conv2D(64,kernel_size=(3,3),padding='valid',activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))

#C_L3
model.add(Conv2D(128,kernel_size=(3,3),padding='valid',activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))


model.add(Flatten())

model.add(Dense(128,activation='relu'))
model.add(Dense(64,activation='relu'))
model.add(Dense(1,activation='sigmoid'))#bcz binary classification,,so 1 node at o.p layer

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 254, 254, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 127, 127, 32)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 125, 125, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 62, 62, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 60, 60, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 30, 30, 128)       0

In [13]:
#Total params: 14847297 (56.64 MB)-->1.48 cr parameters nad that are trainable 

In [14]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
myhistory=model.fit(train_ds,epochs=10,validation_data=validation_ds)
#if u run this in Jupyter notebook ,,then it takes lot-lot of time
#ie;32 batches  means 20k/32==>625 batches are there ,,and also 10 epochs
#in every epochs u have to train 625 batches,,,and for ur kind info -->to train 1 epoch completely require more than 2hrs
#so use Google colab,,and set runtime on GPU,,therefore with 1min ,,all 10 epochs completes